In [ ]:
# This takes photometry of stars through apertures arranged on a grid which moves
# to compensate for poor telescope guiding

# Created 2021 Aug. 20 by E.S.

In [1]:
import pandas as pd
import numpy as np
#from astropy.stats import sigma_clipped_stats
#from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import glob
import os
from numpy.linalg import inv
import svdt

%matplotlib qt

In [2]:
stem_data = "/Users/bandari/Documents/git.repos/kriz_projects/data/2021-08-04/00_dark_subtracted_flat_fielded/"
stem_star_positions = "/Users/bandari/Documents/git.repos/kriz_projects/notebooks_for_development/output_other/"

In [3]:
# names of files

file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

file_names_pos = sorted(glob.glob(stem_star_positions + "V1432_Aql-*dat"))

# file of measured displacements between images
file_name_disp = "displacements_v1432_aql.csv"

In [6]:
# machinery to find rotation and translation with SVD:

# Matrix of empirically found positions        F: [ [x_1' y_1' z_1'=0], [x_2' y_2' z_2'=0], [x_3' y_3' z_3'=0] ]
# Rotation matrix                              R
# Matrix of estimated, translated positions    T: [ [x_1 y_1 z_1=0], [x_2 y_2 z_2=0], [x_3 y_3 z_3=0] ]

# fake data
T_estimate = np.array([[0.5, 0.5*np.sqrt(3)/2., 0.], [-0.5*np.sqrt(3)/2., 0.5, 0.], [-0.5, -0.5*np.sqrt(3)/2.,0.]])
F = np.array([[0, 1., 0],[-1, 0, 0],[0, -1, 0]])


R, L, RMSE = svdt.svd(A=T_estimate, B=F)

print(R)
print(L)
print(RMSE)

[[ 0.65465367 -0.75592895  0.        ]
 [ 0.75592895  0.65465367  0.        ]
 [ 0.          0.          1.        ]]
[-1.12854057e-01 -2.77555756e-17  0.00000000e+00]
0.18428990404026965


In [75]:
L

array([ 0.70601613, -0.94893617,  0.        ])

In [72]:
coord_mapping_brightest.to_array()

AttributeError: 'DataFrame' object has no attribute 'to_array'

In [ ]:
F_matrix = np.array(())

In [8]:
def coords_brightest(star_positions_scrambled_ids_pass, num_rank):
    '''
    Return the coordinates of the brightest stars in the dataframe (column "Flux")
    
    INPUTS:
    star_positions_scrambled_ids_pass: DataFrame with xcentroid, ycentroid, and flux info
    num_rank: Nth brightest stars returned (i.e., 3 -> coordinates of 3 brightest are returned)
    
    OUTPUTS:
    2D array of x,y coordinates of the Nth brightest stars
    '''
    
    brightest_fluxes = np.sort(star_positions_scrambled_ids_pass["flux"])[-int(num_rank):]
    #print("brightest")
    #print(brightest_fluxes)
    # pick table rows where stars are at least as bright as the third-brightest star
    idx_cond = star_positions_scrambled_ids_pass["flux"]>=brightest_fluxes[0]
    # get their positions
    x_brights = star_positions_scrambled_ids_pass["xcentroid"][idx_cond]
    y_brights = star_positions_scrambled_ids_pass["ycentroid"][idx_cond]
    #print(star_positions_scrambled_ids_pass[idx_cond])
    coords_empir_brights = zip(x_brights,y_brights)
    
    return list(coords_empir_brights)

In [69]:
def match_stars(predicted_coords_pass, empirical_coords_pass):
    '''
    Matches up predicted and empirical coordinates of bright stars
    
    INPUTS:
    predicted_coords_pass: list of predicted positions of the brightest stars in the initial frame
    empirical_coords_pass: list of empirical coordinates of LOTS of stars in the current frame
    
    OUTPUTS:
    df_map: DataFrame with mapping of predicted and true coordinates
    '''
    
    # convert inputs to dataframes
    df_pred = pd.DataFrame(predicted_coords_pass, columns=["x_pred", "y_pred"])
    df_empir = pd.DataFrame(empirical_coords_pass, columns=["x_empir", "y_empir"])
    
    # initialize columns of 'true' coordinates to predicted dataframe
    df_pred["x_true"] = np.nan
    df_pred["y_true"] = np.nan

    
    # pluck out the 'true' coordinates from the empirical data
    for num_bright in range(0,len(df_pred)):
        # condition is "where difference in coordinates is less than N=3 pixels"
        idx_cond = np.sqrt(
                        np.add(
                            np.power(np.subtract(df_pred["x_pred"][num_bright],df_empir["x_empir"]),2.),
                            np.power(np.subtract(df_pred["y_pred"][num_bright],df_empir["y_empir"]),2.)
                            )
                        ) < 3
        
        # populate the 'true' columns of the DataFrame of predicted values
        df_pred["x_true"][num_bright] = df_empir["x_empir"].loc[idx_cond].values[0]
        df_pred["y_true"][num_bright] = df_empir["y_empir"].loc[idx_cond].values[0]
        
    df_map = df_pred.copy(deep=True)
        
    return df_map

In [9]:
# read in image displacements for ALL stars in ALL the frames
disp_images = pd.read_csv(stem_star_positions+file_name_disp)
# read in first image's star locations (but with scrambled IDs)
initial_star_pos = pd.read_csv(file_names_pos[0], delim_whitespace=True)
#print(initial_star_pos)

# find the coordinates of the brightest three stars in the initial image;
# we will use this to figure out the net translation and rotation of successive frames
initial_coords_empir_brights = coords_brightest(initial_star_pos,num_rank=3) 
#print(initial_coords_empir_brights)

In [70]:
## ## START FOR-LOOP
for num_star_file in range(0,2200,100): #1200,2200 len(file_names_stars)):
    print("num star file")
    print(num_star_file)

    # make copy of DataFrame of initial stars (we will change the x,y coords to make a prediction)
    predicted_star_pos = initial_star_pos.copy(deep=True)

    # read in current star image
    star_image = fits.open(file_names_stars[num_star_file])[0].data

    # read in empirically-found positions and photometry of ALL stars in this frame (but with scrambled IDs)
    star_info_scrambled_ids = pd.read_csv(file_names_pos[num_star_file], delim_whitespace=True)

    # retrieve (x,y) linear displacement vectors found independently for ALL stars in that frame, as was 
    # found by cross-correlating images
    idx = disp_images["file_name"]==str(os.path.basename(file_names_stars[num_star_file]))
    x_disp = disp_images["x_off_pix"].where(idx).dropna().values[0]
    y_disp = disp_images["y_off_pix"].where(idx).dropna().values[0]

    # shift aperture grid of first (i.e., [0]) array of images by the linear displacement; this is a first-order 
    # correction to fit the current frame (n.b. we use this grid and translate it in order to preserve the ID 
    # information of the stars; otherwise, if we re-find the stars in each frame we don't know which is which)
    #print("print(predicted_star_pos) before")
    #print(predicted_star_pos)
    predicted_star_pos["xcentroid"] = initial_star_pos["xcentroid"]-x_disp
    predicted_star_pos["ycentroid"] = initial_star_pos["ycentroid"]-y_disp
    predicted_coords = list(zip(predicted_star_pos["xcentroid"],predicted_star_pos["ycentroid"]))

    # find the predicted coordinates (in this image) of the brightest three stars (as measured in the initial image)
    print("---")
    predicted_coords_brights = coords_brightest(predicted_star_pos,num_rank=3)
    print("predicted_coords_brights")
    print(predicted_coords_brights)
    print(type(predicted_coords_brights))
    # find the empirical coordinates of the brightest *twenty* stars in this image
    # (here we choose twenty, and from those we'll match the correct three, because the brightest
    # three stars in the first image are not necessarily the brightest three in every image)
    empirical_coords_brights = coords_brightest(star_info_scrambled_ids,num_rank=30)
    print("empirical_coords_brights")
    print(empirical_coords_brights)
    print(type(empirical_coords_brights))
    print("---")
    
    # match the stars by finding the sets of predicted and empirical coordinates that are within 3 (or so) pixels
    coord_mapping_brightest = match_stars(predicted_coords_brights, empirical_coords_brights)
    
    # use SVD decomposition to find the translation and rotation to minimize the residuals
    predicted_array = np.array([[coord_mapping_brightest["x_pred"][0], coord_mapping_brightest["y_pred"][0], 0.], 
                           [coord_mapping_brightest["x_pred"][1], coord_mapping_brightest["y_pred"][1], 0.], 
                           [coord_mapping_brightest["x_pred"][2], coord_mapping_brightest["y_pred"][2], 0.]])
    true_array = np.array([[coord_mapping_brightest["x_true"][0], coord_mapping_brightest["y_true"][0], 0.], 
                           [coord_mapping_brightest["x_true"][1], coord_mapping_brightest["y_true"][1], 0.], 
                           [coord_mapping_brightest["x_true"][2], coord_mapping_brightest["y_true"][2], 0.]])
    R, T, RMSE = svdt.svd(A=predicted_array, B=true_array)

    ## FYI
    '''
    apertures = CircularAperture(predicted_coords, r=4.)
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    plt.show()
    '''

    '''
    print("initial star pos")
    print(initial_star_pos)
    vectors_translation = np.transpose((np.subtract(initial_star_pos['xcentroid'],x_disp), 
                              np.subtract(initial_star_pos['ycentroid'],y_disp)))

    #print(vectors_translation.shape)
    print("vectors translation")
    print(vectors_translation)
    '''
    '''

    # get predicted (simple translation) coords of the brightest stars in the first frame
    # get empirical coords of the brightest stars in this frame
    df_vectors_translation = pd.DataFrame(vectors_translation, columns=["x_transl", "y_transl"])
    coords_predic_brights = coords_brightest(df_positions_translation)



    # take the three brightest ones

    # get empirical coords of the brightest stars in this frame
    #print(type(star_positions_scrambled_ids))
    coords_empir_brights = coords_brightest(star_positions_scrambled_ids)
    '''
    # get residuals from mapping of the predicted with the empirical coordinates
    # (then we know which star is which)
    #print(initial_coords_empir_brights)
    #print(coords_empir_brights)
    #print(coords_predic_brights)




    #print(star_positions_scrambled_ids.where[star_positions_scrambled_ids["xcentroid"]==brightest_fluxes[0]])["xcentroid"]

    '''


    ## ## fcn to find the positions of the three brightest stars closest to their estimated positions, 
    ## ## and with a brightness that is consistent (brightest stars have IDs 26, 272, 99 [I believe; check])

    file_names_pos
    '''
    ## ## insert SVD functionality here to find residual translation and rotation

    ## ## make the residual transformation to the aperture locations




    '''
    print(positions)

    apertures = CircularAperture(positions, r=4.)


    phot_table = aperture_photometry(star_image, apertures)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output


    ## ## START TEST
    # convert to DataFrame
    df_phot = phot_table.to_pandas()

    # remove rows with coordinates that put the stars outside the 512x512 frame, or close to edge
    edge_buffer = 15 # pix
    #phot_table = phot_table[~mask]
    idx_edge_xs = np.logical_or(df_phot["xcenter"] > np.subtract(512,edge_buffer), df_phot["xcenter"] < edge_buffer)
    idx_edge_ys = np.logical_or(df_phot["ycenter"] > np.subtract(512,edge_buffer), df_phot["ycenter"] < edge_buffer)
    df_phot_drop_edges = df_phot.drop(df[np.logical_or(idx_edge_xs,idx_edge_ys)].index, inplace = False)

    # sort: brightest stars first
    df_phot_drop_edges = df_phot_drop_edges.sort_values("aperture_sum", axis=0, ascending=False)

    print(df_phot)
    print(df_phot_drop_edges)

    ## ## END TEST
    '''
    # write text file with aperture photometry
    '''
    text_file_name = "output_other/aper_photom_"+str(os.path.basename(file_names_stars[num_star_file])).split(".")[-2]+".dat"
    ascii.write(phot_table, text_file_name, overwrite=False)
    print("Wrote out "+text_file_name)

    # write FYI plot
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    '''


    ## ## END FOR-LOOP

num star file
0
---
predicted_coords_brights
[(377.49692999999996, 42.27124), (437.26032999999995, 158.11151999999998), (476.31222, 462.25875999999994)]
<class 'list'>
empirical_coords_brights
[(0.18494507, 4.425652599999999), (77.49881500000001, 10.993477), (24.935910999999997, 12.60379), (6.989014, 14.224342000000002), (377.49692999999996, 42.27124), (164.38138, 44.6132), (403.7978, 78.796607), (271.96066, 113.38598), (447.31089000000003, 136.51035), (437.26032999999995, 158.11151999999998), (122.90063, 186.92496), (403.49201, 225.20182000000003), (473.56820999999997, 225.7844), (479.68217999999996, 250.64405), (255.50755, 251.43361000000002), (399.30348, 283.66726), (165.03049, 312.38501), (47.60565, 331.42663999999996), (57.849678000000004, 332.36568), (267.1154, 348.84882999999996), (326.01628999999997, 352.69773), (122.97466999999999, 364.77616), (393.33423, 377.29175), (102.19606999999999, 378.92922999999996), (273.97594, 394.57069), (393.15572000000003, 435.6156), (139.75766000

num star file
800
---
predicted_coords_brights
[(338.79693, 45.62124), (398.56032999999996, 161.46151999999998), (437.61222000000004, 465.60875999999996)]
<class 'list'>
empirical_coords_brights
[(38.943505, 14.091209), (481.89870999999994, 16.210461), (339.06131, 45.487097), (125.79523, 47.794989), (505.56922000000003, 88.968825), (68.195779, 103.55717), (233.50623, 116.56963), (408.64709, 140.06222), (496.86136, 152.02005), (398.5912, 161.61659), (490.85425, 173.46081999999998), (84.281564, 190.03793000000002), (492.06385, 224.2032), (364.74627000000004, 228.68597999999997), (434.84756, 229.42291), (440.92235999999997, 254.35823), (216.71187, 254.79664), (360.61138, 287.1129), (8.975122599999999, 334.4969), (19.105329, 335.60722999999996), (228.40002, 352.15676), (287.2813, 356.14075), (84.32579100000001, 367.88575), (487.97202999999996, 369.70597999999995), (235.23642999999998, 397.87646), (500.72809000000007, 437.07304000000005), (100.92004, 443.6339), (65.930589, 463.24617), (437.

num star file
1900
---
predicted_coords_brights
[(318.52693, 56.501239999999996), (378.2903299999999, 172.34151999999997), (417.34222, 476.48875999999996)]
<class 'list'>
empirical_coords_brights
[(18.954371, 24.344584), (461.78747000000004, 27.350869), (319.01076, 56.416847), (105.78606, 58.267430000000004), (496.02848, 76.003093), (485.32125, 100.11685), (48.051574, 114.03007), (213.4104, 127.24805), (388.44012000000004, 150.9849), (476.58403, 163.1345), (378.30482, 172.46995), (470.61262999999997, 184.54352), (64.071001, 200.40435), (471.8372, 235.37677000000002), (414.5781, 240.39649), (196.5644, 265.51332), (420.57565, 265.32436), (340.1718, 297.94806), (105.96806000000001, 326.0351), (0.060773447, 345.72587999999996), (207.91001, 362.84896000000003), (266.86054, 366.9387), (63.73648000000001, 378.32459), (467.5366, 380.82769), (214.67982999999998, 408.58994), (480.09405999999996, 448.19665), (80.409987, 454.08925999999997), (45.507226, 473.57592), (416.74597, 476.84999000000005),

In [10]:
from numpy.linalg import inv
a = np.array([[1., 2.], [3., 4.]])
ainv = inv(a)

In [12]:
print(a)

[[1. 2.]
 [3. 4.]]


In [11]:
np.allclose(np.dot(a, ainv), np.eye(2))
np.allclose(np.dot(ainv, a), np.eye(2))

True

In [42]:
'''
# for plotting
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)
'''

In [44]:
# star images
file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

# darks
file_names_darks = sorted(glob.glob(stem_data + "d*fit"))

# flats
file_names_flats = sorted(glob.glob(stem_data + "flat*fit"))